#### (1) 가산 잡음 처리

In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

image = cv2.imread('../data/Lena.png').astype(np.float32) / 255
avg = np.zeros_like(image)
n = 0

for i in range(20):
    noise = np.random.randn(*image.shape).astype(np.float32) * 0.25 # 표준편차 0.25
    noised = (image + noise).clip(0,1)
    avg += noised
    n += 1
    psnr1 =cv2.PSNR(image, noised,1)
    psnra =cv2.PSNR(image, avg/n,1) # 합쳐서 평균을 내면 잡음이 점점 줄어든다 -> 사진은 원본 그대로 사용 but 잡음은 random적이므로 화소별로 더해주거나 빼줄텐데 잡음끼리 계산되어서 잡음이 줄어들기 때문이다.
    cv2.imshow('noise reduction: {:0.2f} -> {:0.2f} dB'.format(psnr1,psnra), np.hstack([noised,avg/n]))
    cv2.waitKey()
    cv2.destroyAllWindows()


#### (2) IIR Filter

In [5]:
import cv2
import numpy as np

image = cv2.imread('../data/Lena.png').astype(np.float32) / 255
avg = np.zeros_like(image)
n = 0
filtered = np.zeros_like(image)

alpha = 0.9 # 기존의 것을 90%남기고, 나머지 10%를 새로 들어온 영상을 이용

for i in range(30):
    noise = np.random.randn(*image.shape).astype(np.float32) * 0.25
    noised =(image + noise).clip(0,1)
    filtered = filtered * alpha + noised * (1-alpha) 
    # filtered = cv2.normalize(filtered,None,0,1,cv2.NORM_MINMAX)
    avg += noised
    n += 1

    psnr1 =cv2.PSNR(image, noised,1)
    psnra =cv2.PSNR(image, avg/n,1)
    psnrf =cv2.PSNR(image, filtered,1)
    cv2.imshow('noise reduction: {:0.2f} -> {:0.2f} -> {:0.2f} dB'.format(psnr1,psnra,psnrf), \
        np.hstack([noised,avg/n,filtered]))
    cv2.waitKey()
    cv2.destroyAllWindows()

#### (3) Spatial Filter 1 - Moving Average

In [6]:
image = cv2.imread('../data/Lena.png').astype(np.float32) / 255
noise = np.random.randn(*image.shape).astype(np.float32) * 0.1
noised = (image + noise).clip(0,1)

blur = cv2.blur(noised,(11,11)) # 숫자를 줄이면 잡음은 덜 줄지만, 조금 덜 blur된다!
cv2.imshow('moving average', np.hstack([noised,blur]))
cv2.waitKey()
cv2.destroyAllWindows()

#### (4) Spatial Filter 2 - Gaussian Blur

**Gaussian Kernel**

$G_i = \alpha * e^-{\frac{(i-(ksize-1)/2)^2}{2*\sigma^2}}, i=0,1,...,ksize-1$

$\alpha$: scale factor chosen s.t. $\sum_i G_i = 1$

**kernel = getGaussianKernel(ksize, sigma, ktype=CV_64F)**

-`ksize` : Aperture size. It should be odd ( `ksize mod 2 = 1` ) and positive.

-`sigma` : Gaussian standard deviation. If it is non-positive, it is computed from ksize as $\sigma = 0.3*((ksize-1)*0.5 - 1) + 0.8$.

-`ktype` : Type of filter coefficients. It can be CV_32F or CV_64F .

In [7]:
gauss = cv2.GaussianBlur(noised, (11, 11), 0)

kernel = cv2.getGaussianKernel(11, 0)
filtered = cv2.filter2D(noised, -1, kernel)

cv2.imshow('filtered', np.hstack([blur,gauss,filtered]))
cv2.waitKey()
cv2.destroyAllWindows()

#### (5) Spatial Filter 3 - Median Filter

In [8]:
median_blur = cv2.medianBlur(noised, 5)
cv2.imshow('filtered', np.hstack([blur,median_blur]))
cv2.waitKey()
cv2.destroyAllWindows()

#### (6) Spatial Filter 4 - Bilateral Filter

In [12]:
bilat = cv2.bilateralFilter(noised,11,0,0)
cv2.imshow('filtered', np.hstack([blur,bilat]))
cv2.waitKey()
cv2.destroyAllWindows()

In [10]:
cv2.imshow('MA-Gauss-Bilat-Median (clockwize)', np.vstack([np.hstack([blur,gauss]),np.hstack([median_blur,bilat])]))
cv2.waitKey()
cv2.destroyAllWindows()

#### (7) Adaptive Wiener Filter

In [11]:
import cv2
import numpy as np

image = cv2.imread('../data/Lena.png').astype(np.float32) / 255
noise = np.random.randn(*image.shape).astype(np.float32) * 0.1
noised = (image + noise).clip(0,1)
filtered = noised.copy()

y, x = image.shape[:2]

for i in range(11,y-11):
    for j in range(11,x-11):
        for k in range(3):
            block = noised[i-11:i+12,j-11:j+12,k]
            mf = np.mean(block)
            sig = np.std(block)
            filtered[i,j,k] = mf + sig**2/(sig**2+0.01)*(noised[i,j,k]-mf)

cv2.imshow('filtered', np.hstack([noised,filtered]))
cv2.waitKey()
cv2.destroyAllWindows()

#### (8) Measure of matching

In [11]:
methods = ['cv2.TM_CCOEFF', 'cv2.TM_CCORR','cv2.TM_SQDIFF', 
            'cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF_NORMED']

import cv2
import numpy as np
import matplotlib.pyplot as plt

image = cv2.imread('../data/Lena.png').astype(np.float32) / 255
noise = np.random.randn(*image.shape).astype(np.float32) * 0.1
noised = (image + noise).clip(0,1)

median_blur = cv2.medianBlur(noised, 5)
bilat = cv2.bilateralFilter(noised,11,0,0)

psnr = cv2.PSNR(image, median_blur, 1)
print(psnr,'dB')

for k in range(6):
    res = cv2.matchTemplate(image, median_blur, eval(methods[k]))
    if k <= 3:
        res = res/(image.shape[0]*image.shape[1]*image.shape[2])
    if k == 2:
        psnr1 = -10*np.log10(res[0][0])
        print(psnr1,'dB', end=' ')
    print(methods[k], res)


27.792012816405276 dB
cv2.TM_CCOEFF [[0.0310449]]
cv2.TM_CCORR [[0.3047682]]
27.791971791655655 dB cv2.TM_SQDIFF [[0.00166266]]
cv2.TM_CCOEFF_NORMED [[1.23873368e-06]]
cv2.TM_CCORR_NORMED [[0.99728286]]
cv2.TM_SQDIFF_NORMED [[0.00544066]]
